# Probe Training Tutorial

We use transformer lens to load models, embeddings etc. 
We use regex and nltk to get tokens matching natural language english words which start with particular letters.
We train linear probes on a balanced dataset, and track binary classification quality. 

For now, this is almost all "under the hood".

To do:
- Find optimal hyper-parameters. 
- Explore internal probes. 
- Understand the directions in more detail.

In [4]:
import sys 
sys.path.append("..")

from transformer_lens.HookedTransformer import HookedTransformer
from transformer_lens.loading_from_pretrained import OFFICIAL_MODEL_NAMES

OFFICIAL_MODEL_NAMES[1:10]
model = HookedTransformer.from_pretrained("EleutherAI/gpt-j-6B")

from src.probe_training import all_probe_training_runner

vocab = model.tokenizer.get_vocab()
probe_weights_tensor = all_probe_training_runner(
    embeddings=model.W_E.detach(),
    vocab=vocab,
    alphabet="CAR",
    criteria_mode="starts",
    probe_type="linear",
    num_epochs=4,
    batch_size=32,
    learning_rate=0.005,
    train_test_split=0.95,
    rebalance=True,
    use_wandb=True,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/gpt-j-6B into HookedTransformer


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jbloom. Use `wandb login --relogin` to force relogin


Epoch 1 / 4 | Loss: 0.3835 | Precision: 0.7500 | Recall: 1.0000: 100%|██████████| 771/771 [00:01<00:00, 673.92it/s]
Epoch 2 / 4 | Loss: 0.2803 | Precision: 0.9286 | Recall: 0.9286: 100%|██████████| 771/771 [00:00<00:00, 883.25it/s]
Epoch 3 / 4 | Loss: 0.2091 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 882.35it/s]
Epoch 4 / 4 | Loss: 0.1323 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 891.90it/s]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


...

eval_metrics/accuracy,▁▃▅█
eval_metrics/f1,▁▃▅█
eval_metrics/loss,█▄▂▁
eval_metrics/precision,▁▃▄█
eval_metrics/recall,▁▁▁▁
metrics/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_metrics/loss,██▇▇▆▆▅▅▅▄▄▄▄▃▄▄▃▃▃▃▂▂▂▂▃▂▃▁▂▂▂▂▂▂▁▁▁▁▁▁
training_metrics/precision,▁▁▁▆███▇███▆▆█▇▅█▇███████▇▇███▇█████████
training_metrics/recall,████▁███████████████████████████████████
eval_metrics/accuracy,98.38213
eval_metrics/f1,0.76923


Epoch 1 / 4 | Loss: 0.3781 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 864.00it/s]
Epoch 2 / 4 | Loss: 0.2011 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 865.84it/s]
Epoch 3 / 4 | Loss: 0.1833 | Precision: 1.0000 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 897.24it/s]
Epoch 4 / 4 | Loss: 0.1082 | Precision: 0.9500 | Recall: 1.0000: 100%|██████████| 771/771 [00:00<00:00, 884.01it/s]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval_metrics/accuracy,▁▄▇█
eval_metrics/f1,▁▄▆█
eval_metrics/loss,█▄▂▁
eval_metrics/precision,▁▃▆█
eval_metrics/recall,▁▁▁▁
metrics/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_metrics/loss,██▇▇▆▆▅▄▄▄▄▄▃▃▄▃▃▃▃▃▃▂▂▂▂▃▂▂▂▂▁▁▂▁▂▁▁▁▁▁
training_metrics/precision,█▅█▅▁██▅███▅█████▅███▅███▆██████████████
training_metrics/recall,▁███████████████████████████████████████
eval_metrics/accuracy,99.30663
eval_metrics/f1,0.85246


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


# To Visualize Predictions, we can do something like this.

In [6]:
from src.dataset import get_letter_dataset

train_loader, test_loader = get_letter_dataset(
    criterion="starts",
    target="A",
    embeddings=model.W_E,
    vocab=vocab,
    batch_size=16,
    rebalance=True,
)

for batch in train_loader:
    print(batch[0][0])
    print(batch[0][1].shape)
    print(batch[1])
    # get proportions of positive and negative labels
    print(sum(batch[1]) / len(batch[1]))
    break

import plotly.express as px
import pandas as pd


def plot_batch_predictions(words, embeddings, probe):
    predictions = probe(embeddings).squeeze()
    predictions = predictions.squeeze()

    # make a dataframe
    df = pd.DataFrame(
        {
            "words": words,
            "predictions": predictions.cpu().detach().numpy(),
            "labels": batch[1].cpu().detach().numpy(),
        }
    )
    # convert labels to bool
    df["labels"] = df["labels"].astype(bool)
    # sort by predictions
    df = df.sort_values(by=["predictions"], ascending=True)

    fig = px.bar(
        df,
        y="words",
        x="predictions",
        color="labels",
        title="Probe weights for letter A",
        labels={
            "x": "Word",
            "y": "Probe Logit",
            "labels": "Starts with A",
        },
        text_auto="0.2f",
        text="words",
        height=800,
        # width=1200,
        template="plotly_white",
        category_orders={"index": df.index[::-1]},
    )
    # update the hover template to make the word much larger
    fig.update_traces(hovertemplate="<b>%{y}</b><br><br>%{x}")

    return fig


reverse_vocab = {v: k for k, v in vocab.items()}
words = [reverse_vocab[i.item()].strip("Ġ") for i in batch[0][0]]
plot_batch_predictions(words, batch[0][1], probe_weights_tensor["A"])

tensor([-0.0102, -0.0008, -0.0030,  ..., -0.0065,  0.0065,  0.0147],
       device='cuda:0', grad_fn=<SelectBackward0>)
torch.Size([4096])
tensor([False, False, False, False, False, False, False, False,  True, False,
         True,  True,  True, False, False,  True], device='cuda:0')
tensor(0.3125, device='cuda:0')


KeyError: -0.010202172212302685